In [1]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import numpy as np
import re
import random
from random import randint
import time #delay
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import Select
from tqdm import tqdm
 
print(os.getcwd())

c:\Users\maimranj\Desktop\BIMB\backend\legisltaion\puA


In [14]:
driver=webdriver.Edge()

publication_date = []
pu_no = []
title = []
status_of_legislation = []
related_legislation = []
date_of_commencement = []
pdf_link = []

url = 'https://lom.agc.gov.my/subsid.php?type=pua'
driver.get(url)
time.sleep(random.randint(5, 10))

# Loop through all 589 pages
for page in range(1, 590):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    pua_list = soup.find('table', attrs={'class': 'table table-striped table-bordered dataTable'}).find_all('tr')[1:]

    for pua in pua_list:
        cols = pua.find_all('td')

        if len(cols) >= 6:
            publication_date.append(cols[0].get_text(strip=True))
            pu_no.append(cols[1].get_text(strip=True))

            title_text = cols[2].get_text(separator="\n", strip=True)
            title.append(title_text)
            
            status_of_legislation.append(cols[3].get_text(strip=True))
            related_legislation.append(cols[4].get_text(strip=True))
            date_of_commencement.append(cols[5].get_text(strip=True))

            pdf_links = cols[6].find_all('a', href=True)
            pdf_link.append(f"https://lom.agc.gov.my{pdf_links[0]['href']}" if pdf_links else 'N/A')

    if page < 589:
        select_element = Select(driver.find_element(By.CSS_SELECTOR, 'div.dataTables_paginate select'))
        select_element.select_by_value(str(page + 1))
        time.sleep(random.randint(5, 10))

pua_df = pd.DataFrame({
    'Publication Date': publication_date,
    'P.U. No': pu_no,
    'Title': title,
    'Status of Legislation': status_of_legislation,
    'Related Legislation': related_legislation,
    'Date of Commencement': date_of_commencement,
    'PDF Link': pdf_link
})

pua_df

,Publication Date,P.U. No,Title,Status of Legislation,Related Legislation,Date of Commencement,PDF Link
0,28/11/2024,P.U. (A) 371/2024,PERINTAH CUKAI PERKHIDMATAN (PENGENAAN CUKAI B...,Amendment,P.U. (A) 212/2018,01/12/2024,https://lom.agc.gov.my../../../ilims/upload/po...
1,28/11/2024,P.U. (A) 370/2024,PERINTAH CUKAI PERKHIDMATAN \n(PENGENAAN CUKAI...,Principal,,01/12/2024,https://lom.agc.gov.my../../../ilims/upload/po...
2,27/11/2024,P.U. (A) 369/2024,PERINTAH CUKAI JUALAN (PENGENAAN CUKAI JUALAN ...,Amendment,P.U. (A) 206/2018,01/12/2024,https://lom.agc.gov.my../../../ilims/upload/po...
3,27/11/2024,P.U. (A) 368/2024,PERINTAH JALAN PERSEKUTUAN (MALAYSIA BARAT) (P...,Amendment,P.U. (A) 401/1989,29/11/2024,https://lom.agc.gov.my../../../ilims/upload/po...
4,27/11/2024,P.U. (A) 367/2024,PERINTAH CUKAI PENDAPATAN (PENGECUALIAN) \n(NO...,Amendment,P.U. (A) 142/2022,-,https://lom.agc.gov.my../../../ilims/upload/po...
...,...,...,...,...,...,...,...
5882,,P.U.(A) 97/1976,MODIFICATION OF LAWS (CRIMINAL PROCEDURE) (SAB...,,,,https://lom.agc.gov.my../../../ilims/upload/po...
5883,,P.U. (A) 42/1974,FEDERAL CAPITAL (CITY AREA) ORDER 1974\nFEDERA...,,,,https://lom.agc.gov.my../../../ilims/upload/po...
5884,,P.U. (A) 209/1971,NOTIFICATION UNDER SECTION 3 OF THE TITLES OF...,,,,https://lom.agc.gov.my../../../ilims/upload/po...
5885,,P.U. (A) 238/1969,ESSENTIAL (MODIFICATIONS OF IMMIGRATION LAWS)...,,,,https://lom.agc.gov.my../../../ilims/upload/po...


In [15]:
driver.close()

In [16]:
pua_df.to_csv('pu(A).csv')